# Nature Poems

In [1]:
import numpy as np
import pandas as pd 
#pd.set_option('display.max_columns', None)
#pd.set_option('display.width', 1000)
import random
import sys
import io
from bs4 import BeautifulSoup
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.optimizers import RMSprop
import requests as rq

Using TensorFlow backend.


In [2]:
#extract the nature poems from first dataset
df = pd.read_csv('poems.csv') # load dataset
poems = df['content']
naturePoems1 = poems.loc[df['type'] == 'Nature']

In [3]:
naturePoems1.head()

19    Why didst thou promise such a beauteous day,\r...
20    The welcome Sun from sea Freake is returned,\r...
21    I met a courtier riding on the plain,\r\nWell-...
22    Walking the fields a wantcatcher I spied,\r\nT...
23    Fishing, if I a fisher may protest,\r\nOf plea...
Name: content, dtype: object

In [14]:
# loading additional nature poems from poetryfoundation.com dataset
df2 = pd.read_csv('PoetryFoundationData.csv') # load dataset
df2 = df2.dropna()
df2.head() # getting top 5 head

,Unnamed: 0,Title,Poem,Poet,Tags
6,6,\r\r\n Invisible Fish\r\r\n...,\r\r\nInvisible fish swim this ghost ocean now...,Joy Harjo,"Living,Time & Brevity,Relationships,Family & A..."
7,7,\r\r\n Don’t Bother the Ear...,\r\r\nDon’t bother the earth spirit who lives ...,Joy Harjo,"Religion,The Spiritual,Mythology & Folklore,Fa..."
9,9,"\r\r\n [""Hour in which I co...","\r\r\nHour in which I consider hydrangea, a sa...",Simone White,"Living,Parenthood,The Body,The Mind,Nature,Tre..."
16,16,\r\r\n scars\r\r\n ...,\r\r\nmy father’s body is a map\r\r\na record ...,Truong Tran,"The Body,Family & Ancestors"
17,17,\r\r\n what remains two\r\r...,\r\r\nit has long been forgotten this practice...,Truong Tran,"Infancy,Parenthood,The Body"


In [15]:
searchfor = ['Nature']

In [16]:
# getting additional poems that contain the tag Love 
poems2 = df2['Poem']
naturePoems2 = poems2.loc[df2['Tags'].str.contains('|'.join(searchfor))]

In [17]:
naturePoems2.head()


6      \r\r\nInvisible fish swim this ghost ocean now...
9      \r\r\nHour in which I consider hydrangea, a sa...
126    \r\r\n\r\r\n\r\r\n\r\r\nNYTimes headline for S...
129    \r\r\nAll that's left is the shroud\r\r\n     ...
186    \r\r\nYou will leave your home:\r\r\nnothing w...
Name: Poem, dtype: object

In [18]:
# concatenate lovePoems and lovePoems 2 to create more data
naturePoems = naturePoems1 + naturePoems2


In [19]:
#convert nature series to a string
naturePoems = naturePoems.to_string()

In [20]:
# MAPPING CHARACTERS: for nature poems, Make two dictionaries , one to convert chars to ints, the other to convert ints back to chars

chars = sorted(list(set(naturePoems)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [21]:
# process the dataset:
seqlen = 50
step = seqlen

data_X = []
data_y = []

poemLines = [] 

# creates poem lines 
for i in range(0, len(naturePoems) - seqlen - 1, step):
    poemLines.append(naturePoems[i: i + seqlen + 1])
    
# creating x and y data
data_X = np.zeros((len(poemLines), seqlen, len(chars)), dtype=np.bool)
data_Y = np.zeros((len(poemLines), seqlen, len(chars)), dtype=np.bool)

for i, poemLines in enumerate(poemLines):
    for t, (char_in, char_out) in enumerate(zip(poemLines[:-1], poemLines[1:])):
        data_X[i, t, char_indices[char_in]] = 1
        data_Y[i, t, char_indices[char_out]] = 1

In [22]:
# create the model
model = Sequential()
model.add(LSTM(128, input_shape=(seqlen, len(chars)), return_sequences=True))
model.add(Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=RMSprop(learning_rate=0.01), metrics=['categorical_crossentropy', 'accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 50, 128)           97280     
_________________________________________________________________
dense_2 (Dense)              (None, 50, 61)            7869      
Total params: 105,149
Trainable params: 105,149
Non-trainable params: 0
_________________________________________________________________


In [23]:
# train the model
model.fit(data_X, data_Y, batch_size=128, epochs=80)

Epoch 1/80
4452/4452 [==============================] - 7s 2ms/step - loss: 0.9175 - categorical_crossentropy: 0.9175 - accuracy: 0.8332
Epoch 2/80
4452/4452 [==============================] - 6s 1ms/step - loss: 0.4222 - categorical_crossentropy: 0.4222 - accuracy: 0.8899
Epoch 3/80
4452/4452 [==============================] - 11s 2ms/step - loss: 0.3065 - categorical_crossentropy: 0.3065 - accuracy: 0.9156
Epoch 4/80
4452/4452 [==============================] - 10s 2ms/step - loss: 0.2755 - categorical_crossentropy: 0.2755 - accuracy: 0.9168
Epoch 5/80
4452/4452 [==============================] - 8s 2ms/step - loss: 0.2668 - categorical_crossentropy: 0.2668 - accuracy: 0.9174
Epoch 6/80
4452/4452 [==============================] - 7s 2ms/step - loss: 0.2601 - categorical_crossentropy: 0.2601 - accuracy: 0.9179
Epoch 7/80
4452/4452 [==============================] - 6s 1ms/step - loss: 0.2551 - categorical_crossentropy: 0.2551 - accuracy: 0.9183
Epoch 8/80
4452/4452 [=================

4452/4452 [==============================] - 8s 2ms/step - loss: 0.2176 - categorical_crossentropy: 0.2176 - accuracy: 0.9289
Epoch 61/80
4452/4452 [==============================] - 7s 1ms/step - loss: 0.2173 - categorical_crossentropy: 0.2173 - accuracy: 0.9291
Epoch 62/80
4452/4452 [==============================] - 11s 2ms/step - loss: 0.2168 - categorical_crossentropy: 0.2168 - accuracy: 0.9294
Epoch 63/80
4452/4452 [==============================] - 8s 2ms/step - loss: 0.2165 - categorical_crossentropy: 0.2165 - accuracy: 0.9292
Epoch 64/80
4452/4452 [==============================] - 11s 2ms/step - loss: 0.2159 - categorical_crossentropy: 0.2159 - accuracy: 0.9294
Epoch 65/80
4452/4452 [==============================] - 8s 2ms/step - loss: 0.2155 - categorical_crossentropy: 0.2155 - accuracy: 0.9295
Epoch 66/80
4452/4452 [==============================] - 11s 2ms/step - loss: 0.2147 - categorical_crossentropy: 0.2147 - accuracy: 0.9297
Epoch 67/80
4452/4452 [====================

In [24]:
def sample(preds, temperature=1.0):
    """Helper function to sample an index from a probability array."""
    preds = np.asarray(preds).astype('float64')
    preds = np.exp(np.log(preds) / temperature)  # softmax
    preds = preds / np.sum(preds)                #
    probas = np.random.multinomial(1, preds, 1)  # sample index
    return np.argmax(probas)  

In [25]:
def generating_poem(epoch, start_index, diversity, _):
    """Function invoked at end of each epoch. Prints generated text."""
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = start_index
    diversity = diversity

    generated = ''
    poemLines = naturePoems[start_index: start_index + seqlen]
    generated += poemLines
    print('----- Generating with seed: "' + poemLines + '"')
    sys.stdout.write(generated)

    for i in range(400):
        x_pred = np.zeros((1, seqlen, len(chars)))
        for t, char in enumerate(poemLines):
            x_pred[0, t, char_indices[char]] = 1.
            
        preds = model.predict(x_pred, verbose=0)
        next_index = sample(preds[0, -1], diversity)
        next_char = indices_char[next_index]

        poemLines = poemLines[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

In [28]:
generating_poem(80, 100, .25, _)


----- Generating text after Epoch: 80
----- Generating with seed: "              NaN
19                              "
              NaN
19                                                                                                NaN
12305                                                                                                                              NaN
5993                                                                                                                                                         NaN
12624                             
